<a href="https://colab.research.google.com/github/GioLeo23/Comment-Filtering-System/blob/main/Comments_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [5]:
import io
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [6]:
from tensorflow.keras.layers import TextVectorization
df = pd.read_csv(io.BytesIO(uploaded['train.csv']))

X = df['comment_text']
y = df[df.columns[2:]].values
MAX_FEATURES = 200000
vectorizer = TextVectorization(max_tokens=MAX_FEATURES, output_sequence_length=1800, output_mode='int')
vectorizer.adapt(X.values)
vectorized_text = vectorizer(X.values)
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

model = Sequential()
model.add(Embedding(MAX_FEATURES+1, 32))
model.add(Bidirectional(LSTM(32, activation='tanh')))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='sigmoid'))

model.compile(loss='BinaryCrossentropy', optimizer='Adam')
history = model.fit(train, epochs=2, validation_data=val)

Epoch 1/2
6981/6981 ━━━━━━━━━━━━━━━━━━━━ 741s 105ms/step - loss: 0.0847 - val_loss: 0.0465
Epoch 2/2
6981/6981 ━━━━━━━━━━━━━━━━━━━━ 741s 105ms/step - loss: 0.0468 - val_loss: 0.0427


In [9]:
!pip install gradio==3.50

In [14]:
import tensorflow as tf
import gradio as gr
model.save('comment_eval.h5')

model = tf.keras.models.load_model('comment_eval.h5')

def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)

    text = ''
    for idx, col in enumerate(df.columns[2:-1]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)

    return text

interface = gr.Interface(
    fn=score_comment,
    inputs=gr.Textbox(lines=2, placeholder='Comment to score'),
    outputs='text'
)

# Launch the interface
interface.launch(share=True)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 422ms/step
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 3.50.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://9ac6fa36b18a9c07a0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
